# Sistemas Distribuidos de Procesamiento de Datos II

# Práctica final T2 (Parte I)

• Datos: Fichero con tweets del GP MotoGP de Qatar 2014 (ALTO DATABASE).

• Tecnologías: Spark SQL, DataFrames, Jupyter notebook.

• Versión Spark: 2.2.1 o superior.

• Fecha de entrega: Domingo, 6 de mayo de 2018, a las 23:55.

## Librerias

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("Practica SparkSQL MotoGP 2014")
    .getOrCreate() )
sc = spark.sparkContext
print(spark)
print(sc)

<SparkContext master=local[*] appName=Practica SparkSQL MotoGP 2014>


In [2]:
    from pyspark.sql.types import *
    from pyspark.sql.functions import *

## Importamos los datos

Primero definimos el esquema de datos

In [3]:
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

A continuación cargamos los datos utilizando el esquema definido:

In [4]:
events = spark.read.csv("data/DATASET-Twitter-23-26-Mar-2014-MotoGP-Qatar.csv",
                        header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm")
                        #mode="FAILFAST") #

In [5]:
events.count()

257680

Se han cargaado 257680 registros.

Comprobamos que el esquema del dataframe se corresponde con el definido:

In [6]:
events.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Parent_sys_id: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Mentions: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Name_source: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Pub_date: timestamp (nullable = true)
 |-- URLs: string (nullable = true)
 |-- Tipe_action: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Has_link: byte (nullable = true)
 |-- Has_picture: byte (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Activity: long (nullable = true)
 |-- Followers: long (nullable = true)
 |-- Following: long (nullable = true)
 |-- Location: string (nullable = true)



## PREGUNTA A

a) Calcular el número total de retweets por usuario para los 50 usuarios con más mensajes
en la muestra de tweets analizados. Calcular, para cada uno de estos usuarios la media de
enlaces (URLs) enviados por mensaje. (2.5 puntos).

Los 50 usuarios con más mensajes de la muestra:

In [7]:
users_50 = events\
 .groupBy("Source")\
 .agg(count("Id").alias("tweets"))\
 .orderBy("tweets", ascending=False)\
 .limit(50)
users_50.show(50)

+---------------+------+
|         Source|tweets|
+---------------+------+
|     m_azharaji|   486|
|     twitMOTOGP|   401|
|      johnbokke|   297|
|   qatarflights|   283|
|     box_repsol|   267|
|    yolandaa_95|   185|
|    AlessiaPont|   182|
|    motomatters|   169|
|     MM93Lovers|   169|
|     Sonic_Moto|   165|
|  noelia_260797|   157|
|    birtymotogp|   155|
|  trackseven707|   152|
|   crash_motogp|   149|
|    pedrosistas|   148|
|MarcMarquezTeam|   148|
|   MotoFamilyGP|   144|
|   Kay46_MotoGP|   142|
|    blogenboxes|   133|
|     Ciintiia93|   133|
|      JesiPacio|   130|
|     tigrescuba|   125|
|    VAVEL_motor|   124|
|   MotoGPquotes|   123|
|       plusmoto|   119|
|     thalia_26_|   119|
|   iNotaMental_|   116|
|       Cev_Ford|   115|
|  btsportmotogp|   113|
|        salo_98|   113|
| AnaAlvarez9325|   111|
|     kamseunyil|   111|
|    Carlota_147|   109|
|      Luciadp26|   109|
|  keikokoeswadi|   107|
|    FansHM_9323|   107|
|   Paula_Bravo3|   107|


Numero total de retweets por usuario para los 50 usuarios con mas mensajes en la muestra:

In [8]:
events.where(events.Tipe_action == "RT")\
.join(users_50,"Source", how="right")\
.groupBy("Source","Tipe_action")\
.agg(count("Id").alias("retweets"))\
.orderBy("retweets", ascending=False)\
.show(50) # Son 44

+---------------+-----------+--------+
|         Source|Tipe_action|retweets|
+---------------+-----------+--------+
|     twitMOTOGP|         RT|     401|
|      johnbokke|         RT|     289|
|   qatarflights|         RT|     215|
|    AlessiaPont|         RT|     181|
|     MM93Lovers|         RT|     169|
|  trackseven707|         RT|     152|
|  noelia_260797|         RT|     128|
|    yolandaa_95|         RT|     126|
|   MotoGPquotes|         RT|     123|
|     tigrescuba|         RT|     123|
|     Ciintiia93|         RT|     119|
|      JesiPacio|         RT|     118|
|        salo_98|         RT|     104|
|nisaauliarahma5|         RT|     101|
|    Carlota_147|         RT|      94|
|     Sonic_Moto|         RT|      94|
|     kamseunyil|         RT|      93|
|      AsyrafAye|         RT|      92|
|   rohimahfidia|         RT|      89|
|      _montse24|         RT|      83|
|      Luciadp26|         RT|      77|
|     Cantimoto3|         RT|      75|
|   Paula_Bravo3|        

Hay seis twitteros de los 50 con mas mensajes de la muestra que no han hecho ningun retweet.

Media de enlaces URL enviados por mensaje para los 50 usuarios con mas mensajes en la muestra:

In [9]:
# Creamos una UDF para obtener el número de URLs por mensaje:
def string_len_list(string):
    if string is not None and string != '0':
        return len(string.split(','))
    else:
        return 0
string_len_list_udf = udf(string_len_list, IntegerType())

Calculamos la media de URLs sumando el numero total de URL´s incluidas en todos los tweets (TW, RT y MT) de un usuario dividido entre el número total de tweets enviados por dicho usuario:

In [10]:
events.join(users_50,"Source", how = "right")\
.withColumn("numero_URLs", string_len_list_udf(events.URLs))\
.groupBy("Source")\
.agg(sum("numero_URLs").alias("total_URLs"),count("Id").alias("tweets"),(round(sum("numero_URLs")/count("Id"),2)).alias("media_URLs"))\
.orderBy("media_URLs", ascending=False)\
.show(50)

+---------------+----------+------+----------+
|         Source|total_URLs|tweets|media_URLs|
+---------------+----------+------+----------+
|    VAVEL_motor|       130|   124|      1.05|
|     tigrescuba|       127|   125|      1.02|
|   iNotaMental_|       116|   116|       1.0|
|Miisael_Sound07|        93|    93|       1.0|
|AkbarValeLorenz|        93|    93|       1.0|
|    yolandaa_95|       176|   185|      0.95|
|      AsyrafAye|        93|   102|      0.91|
|   qatarflights|       233|   283|      0.82|
|MarcMarquezTeam|       112|   148|      0.76|
|     twitMOTOGP|       298|   401|      0.74|
|      _montse24|        68|    92|      0.74|
|        salo_98|        83|   113|      0.73|
|   MotoFamilyGP|       102|   144|      0.71|
|   MotoGPquotes|        85|   123|      0.69|
|      JesiPacio|        88|   130|      0.68|
|nisaauliarahma5|        67|   101|      0.66|
|   crash_motogp|        96|   149|      0.64|
|  trackseven707|        95|   152|      0.63|
|     MM93Lov

## PREGUNTA B

b) Calcular el número total de mensajes que contienen información de geolocalización en
el campo LOCATION.(2.5 puntos).

Si la variable de Gelocalización "Location" empieza por “ÜT”, esto indica que se proporcionan las coordenadas exactas desde donde se emitió el tweet.

In [11]:
events.where(col("Location").like("ÜT%")).count()

2136

Observamos que hay 5 twitts a mayores que contienen "ÜT" en la variable "Location" pero no necesariamente al comienzo de la misma:

In [12]:
events.where(col("Location").like("%ÜT%")).count() - events.where(col("Location").like("ÜT%")).count()

5

## PREGUNTA C

c) Calcular las 10 cuentas de Twitter que más han sido mencionadas en todo el conjunto de
datos analizados. (2.5 puntos).

En el campo "Mentions" aparecen los usuarios separados por comas presentes en el contenido de aquellos mensajes tipo RT (Retweet) o MT (Mención).

In [13]:
events.where(events.Tipe_action != "TW")\
.select(explode(split(col("Mentions"), ",")).alias("cuentas_mencionadas"))\
.groupBy("cuentas_mencionadas")\
.agg(count("cuentas_mencionadas").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.limit(10).show()

+-------------------+------+
|cuentas_mencionadas|tweets|
+-------------------+------+
|       valeyellow46| 61226|
|      marcmarquez93| 58132|
|             motogp| 38219|
|          lorenzo99| 16802|
|     26_danipedrosa| 12346|
|      alexmarquez23|  6117|
|         box_repsol|  5304|
|          jessansan|  4568|
|                ims|  3325|
|      btsportmotogp|  2479|
+-------------------+------+



## PREGUNTA D

d) Calcular los 10 mensajes más retweeteados y los 10 mensajes que han acumulado más
respuestas en la muestra de datos analizados. Ahora, restringe la búsqueda a los mensajes
en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00. (2.5 puntos).

El campo "Parent_sys_id", si no es nulo ("sin padre"), contiene un ID que relaciona el post con aquel del cual depende por ser un retweet o una respuesta.

Los diez mensajes más retwiteados:

In [14]:
mensajes_RT_10 = events.where((events.Tipe_action == "RT") & (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_RT_10.show()

+------+------+
|    Id|tweets|
+------+------+
|603804|  5515|
|674825|  4995|
|460820|  3322|
|758581|  3236|
|553100|  2879|
|681159|  2406|
|553344|  2266|
|666385|  1774|
|605727|  1691|
|488086|  1646|
+------+------+



Hacemos un join por "Id" con el dataframe original "events" para obtener más información acerca de los 10 mensajes mas retweeteados:

In [15]:
events.join(mensajes_RT_10, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

[Row(Id='603804', Source='marcmarquez93', Body="increíble, aún no me lo creo! / incredible, i can't believe it! "),
 Row(Id='674825', Source='ValeYellow46', Body='what a race!great battles like good old times i had a lot of fun,i think was great also from tv @motogp rocks!'),
 Row(Id='460820', Source='marcmarquez93', Body='hoy comienza el mundial de motogp! / today starts 2014 motogp world championship! :) http://t.co/cwszfauajo'),
 Row(Id='758581', Source='marcmarquez93', Body='bella foto!! @valeyellow46 http://t.co/dmofztgkb8'),
 Row(Id='553100', Source=None, Body=None),
 Row(Id='681159', Source='ValeYellow46', Body='che gara ragazzi!!! grande inizio di stagione! tante battaglie come ai vecchi tempi,mi sono divertito da matti '),
 Row(Id='553344', Source='MotoGP', Body='another awesome win for @marcmarquez93 as he holds off @valeyellow46 with @26_danipedrosa third! #motogp'),
 Row(Id='666385', Source='marcmarquez93', Body='gracias a todos los que me han ayudado en esta recuperación! 

Los diez mensajes que han acumulado más respuestas:

In [16]:
mensajes_MT_10 = events.where((events.Tipe_action == "MT")& (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_MT_10.show()

+------+------+
|    Id|tweets|
+------+------+
|681159|   554|
|605727|   530|
|603804|   511|
|674825|   493|
|460820|   233|
|738975|   210|
|758581|   153|
|488086|   135|
|461756|   125|
|710916|   115|
+------+------+



Hacemos un join por "Id" con el dataframe original "events" para obtener más información acerca de los 10 mensajes que han acumulado un mayor numero de respuestas:

In [17]:
events.join(mensajes_MT_10, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

[Row(Id='681159', Source='ValeYellow46', Body='che gara ragazzi!!! grande inizio di stagione! tante battaglie come ai vecchi tempi,mi sono divertito da matti '),
 Row(Id='605727', Source='lorenzo99', Body="perdonad equipo y afición, hoy me he equivocado yo. / today i've made a mistake. we will try at austin again! http://t.co/fpyc2oo7mp"),
 Row(Id='603804', Source='marcmarquez93', Body="increíble, aún no me lo creo! / incredible, i can't believe it! "),
 Row(Id='674825', Source='ValeYellow46', Body='what a race!great battles like good old times i had a lot of fun,i think was great also from tv @motogp rocks!'),
 Row(Id='460820', Source='marcmarquez93', Body='hoy comienza el mundial de motogp! / today starts 2014 motogp world championship! :) http://t.co/cwszfauajo'),
 Row(Id='738975', Source='lorenzo99', Body='mirad lo que me acaba de llegar...!! adivináis el piloto? / look what just arrived! driver? #idol #happy #collection http://t.co/kpvidbn2j0'),
 Row(Id='758581', Source='marcmarqu

Restringir la busqueda a los mensajes en el intervalo 2014-03-24 04:00 - 2014-03-24 10:00:

El campo "Pub_date" contiene la fecha y hora en la que se ha escrito el mensaje.

In [18]:
mensajes_RT_10_date = events.where(events.Pub_date.between('2014-03-24 04:00' , '2014-03-24 10:00'))\
.where((events.Tipe_action == "RT") & (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_RT_10_date.show()

+------+------+
|    Id|tweets|
+------+------+
|645626|   443|
|603804|   431|
|638320|   368|
|645328|   226|
|605727|   161|
|635916|   140|
|622993|   138|
|553100|   109|
|627447|    89|
|645225|    89|
+------+------+



In [19]:
events.join(mensajes_RT_10_date, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

[Row(Id='645626', Source='JesSanSan', Body="no os recuerda esta imagen a qatar 2013? pues no es un 'deja vu'... estamos en 2014! valentino rossi is on fire! http://t.co/ng5juwbrwl"),
 Row(Id='603804', Source='marcmarquez93', Body="increíble, aún no me lo creo! / incredible, i can't believe it! "),
 Row(Id='638320', Source='JesSanSan', Body='que foto! valentino rossi .@valeyellow46, marc márquez .@marcmarquez93 y dani pedrosa .@26_danipedrosa #qatar #motogp http://t.co/0vbnzmwirb'),
 Row(Id='645328', Source='JesSanSan', Body='valentino rossi @valeyellow46 con cara de satisfacción en el parque cerrado, mientras es felicitado por su equipo #46 http://t.co/n4lhbww3wm'),
 Row(Id='605727', Source='lorenzo99', Body="perdonad equipo y afición, hoy me he equivocado yo. / today i've made a mistake. we will try at austin again! http://t.co/fpyc2oo7mp"),
 Row(Id='635916', Source='AlbiTebaldi', Body='@morsellilinda @valeyellow46 behind the scenes http://t.co/s6zlhcpdfw'),
 Row(Id='622993', Source='

In [20]:
mensajes_MT_10_date = events.where(events.Pub_date.between('2014-03-24 04:00' , '2014-03-24 10:00'))\
.where((events.Tipe_action == "MT")& (events.Parent_sys_id != "sin padre"))\
.groupBy("Parent_sys_id")\
.agg(count("Parent_sys_id").alias("tweets"))\
.orderBy("tweets", ascending=False)\
.withColumnRenamed("Parent_sys_id", "Id")\
.limit(10)
mensajes_MT_10_date.show()

+------+------+
|    Id|tweets|
+------+------+
|603804|    62|
|605727|    46|
|645225|    12|
|645328|    12|
|645626|     9|
|664891|     9|
|622993|     8|
|626650|     7|
|579001|     6|
|646819|     4|
+------+------+



In [21]:
events.join(mensajes_MT_10_date, on = "Id", how="right")\
.select("Id","Source","Body")\
.orderBy("tweets", ascending=False).take(10)

[Row(Id='603804', Source='marcmarquez93', Body="increíble, aún no me lo creo! / incredible, i can't believe it! "),
 Row(Id='605727', Source='lorenzo99', Body="perdonad equipo y afición, hoy me he equivocado yo. / today i've made a mistake. we will try at austin again! http://t.co/fpyc2oo7mp"),
 Row(Id='645225', Source='mflamigni', Body="che gara!! emozionante fino all'ultimo metro!!!!! grande @valeyellow46 !!! e' davvero un onore essere al tuo fianco in queste 'battaglie'"),
 Row(Id='645328', Source='JesSanSan', Body='valentino rossi @valeyellow46 con cara de satisfacción en el parque cerrado, mientras es felicitado por su equipo #46 http://t.co/n4lhbww3wm'),
 Row(Id='645626', Source='JesSanSan', Body="no os recuerda esta imagen a qatar 2013? pues no es un 'deja vu'... estamos en 2014! valentino rossi is on fire! http://t.co/ng5juwbrwl"),
 Row(Id='664891', Source='DennisNoyes', Body='tres carreras magníficas…y ningún problema con los nuevos reglamentos. podemos olvidarnos ya de reglam

## Paramos el contexto de Spark

In [22]:
sc.stop()